<a href="https://colab.research.google.com/github/okj3031402/LLM_study/blob/main/automating_comment_classification_using_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# input_text : "여기 음식은 언제 와도 실망시키지 않아요. 최고!"

system_prompt = """
Act as a classifier that accurately categorizes the sentiment of comments.
Given a user-input comment,
write '1' if the comment is positive, and '0' if the comment is negative.
Output the INTEGER '1' or '0' ONLY, without any other text.
"""

# 함수의 인자를 그대로 user 입력으로 삽입한다.

def llm(input_text):

    completion = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages =
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ]
    )

    response_message = completion.choices[0].message.content

    # "1입니다." 라고 나올 수 있음
    if '1' in response_message:
        return 1 # 긍정

    return 0 # 부정

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x7dafcf24c3a0>)

In [ ]:
import random
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

def classify_text(input_text):

    output = llm(input_text)

    return output

# 파일을 DataFrame으로 로드
df = pd.read_csv('tarr_train.txt', delimiter='\t')

actual_labels = []
predicted_labels = []

total = len(df)

for index, row in df.iterrows():
    comment = row['comment'] # 댓글

    # 예측
    predicted_label = classify_text(comment)

    # 정답
    actual_label = row['label'] # 정답인 긍정/부정

    actual_labels.append(actual_label)
    predicted_labels.append(predicted_label)

    print(f"[{index+1}]/[{total}]")
    print("comment : ", comment)
    print("actual class : ", actual_label)
    print("predicted class : ", predicted_label)
    print("---------------")

    if index > 8:
        break

# 정확도 계산
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Accuracy: {accuracy*100:.2f}%")

# Confusion matrix 계산
cm = confusion_matrix(actual_labels, predicted_labels, labels=[1, 0])
# Confusion matrix 표현
print("\nConfusion Matrix:")
print("         Predicted:")
print("         긍정    부정")
print("Actual")
print("긍정      {:<5}  {:<5}".format(cm[0][0], cm[0][1]))
print("부정      {:<5}  {:<5}".format(cm[1][0], cm[1][1]))

[1]/[300]
comment :  여기 음식은 언제 와도 실망시키지 않아요. 최고!
actual class :  1
predicted class :  1
---------------
[2]/[300]
comment :  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
actual class :  1
predicted class :  1
---------------
[3]/[300]
comment :  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
actual class :  1
predicted class :  1
---------------
[4]/[300]
comment :  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
actual class :  0
predicted class :  0
---------------
[5]/[300]
comment :  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
actual class :  1
predicted class :  1
---------------
[6]/[300]
comment :  메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
actual class :  1
predicted class :  1
---------------
[7]/[300]
comment :  사진과 음식이 너무 달라서 실망했습니다.
actual class :  0
predicted class :  0
---------------
[8]/[300]
comment :  주변에 추천하려고 사진도 많이 찍었어요. 좋아요!
actual class :  1
predicted class :  1
---------------
[9]/[300]
comment :  솔직히...? 맛이 그닥이에요. 리뷰랑 너무 다르네.
actual class :  0
predicted class :  0
---------------
[10]/[300]
comment :  진짜 개꿀맛..ㅠ 다른곳 안가.
a

In [ ]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_sample_submit.txt",
    filename="tarr_sample_submit.txt",
)

('tarr_sample_submit.txt', <http.client.HTTPMessage at 0x7dafcf24ca60>)

In [ ]:
# 파일을 DataFrame으로 로드
df_submit = pd.read_csv('tarr_sample_submit.txt', delimiter='\t')

predicted_labels = []

total = len(df_submit)

# 각 row를 순회하며 코멘트를 분류
for index, row in df_submit.iterrows():
    comment = row['comment']
    predicted_label = classify_text(comment)
    predicted_labels.append(predicted_label)

    print(f"[{index+1}]/[{total}]")
    print("comment : ", comment)
    print("predicted class : ", predicted_label)
    print("---------------")

    if index > 3:
        break

# 예측된 레이블을 DataFrame에 추가
#df_submit['label'] = predicted_labels

# 결과를 tarr_my_submit.txt로 저장
#df_submit[['id', 'comment', 'label']].to_csv('tarr_my_submit.txt', sep='\t', index=False)

[1]/[100]
comment :  완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌
predicted class :  1
---------------
[2]/[100]
comment :  맛있긴 한데 양이 너무 적어서 좀... ㅠ
predicted class :  0
---------------
[3]/[100]
comment :  완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨
predicted class :  1
---------------
[4]/[100]
comment :  한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.
predicted class :  1
---------------
[5]/[100]
comment :  서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.
predicted class :  0
---------------
